In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Install and load all libraries

In [2]:
from fastai.vision.all import *

In [3]:
pd.set_option('display.max_rows', None)

# Data splitting based on clip name

Reading the dataframe from `CSV` file and removing the tangling column

In [21]:
clips_df = pd.read_csv('data/labels.csv')

# dropping dangling columns
clips_df = clips_df.loc[:, ~clips_df.columns.str.contains('^Unnamed')]
clips_df.head()

,clip_name,tools_present
0,clip_000000,"[needle driver, nan, needle driver, cadiere forceps]"
1,clip_000001,"[needle driver, nan, needle driver, cadiere forceps]"
2,clip_000002,"[needle driver, nan, needle driver, cadiere forceps]"
3,clip_000003,"[needle driver, nan, needle driver, cadiere forceps]"
4,clip_000004,"[needle driver, nan, needle driver, cadiere forceps]"


Standardising labels to reflect the output JSON dict() values by removing brackets (`[]`), quotes (`''`), hyphens (`-`), slashes (`\`) and also replacing underscore(`_`) in tools name for white spaces (` `) 


In [22]:
def clean_labels(c): return str([(i).strip().replace(" ", "_").replace("-", "_").replace("/", "_") for i in re.sub(r"[\[\]]",'',c).split(',')]).replace("'","").strip("'").strip("[").strip("]").replace(" ","")
clips_df['tools_present'] = clips_df['tools_present'].apply(clean_labels)

In [24]:
clips_df.head(30)

,clip_name,tools_present
0,clip_000000,"needle_driver,nan,needle_driver,cadiere_forceps"
1,clip_000001,"needle_driver,nan,needle_driver,cadiere_forceps"
2,clip_000002,"needle_driver,nan,needle_driver,cadiere_forceps"
3,clip_000003,"needle_driver,nan,needle_driver,cadiere_forceps"
4,clip_000004,"needle_driver,nan,needle_driver,cadiere_forceps"
5,clip_000005,"needle_driver,nan,needle_driver,cadiere_forceps"
6,clip_000006,"needle_driver,nan,needle_driver,cadiere_forceps"
7,clip_000007,"needle_driver,nan,needle_driver,cadiere_forceps"
8,clip_000008,"needle_driver,nan,needle_driver,cadiere_forceps"
9,clip_000009,"needle_driver,nan,needle_driver,cadiere_forceps"


In [26]:
# Open clips_df in browser and clip videos in finder 
# Start reviewing video after the last number used in the following dictionary
# Add videos to the list if there is any disparity in labelling

corrects=[
    {'from': 'clip_000000', 'to':'clip_000026', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000028', 'to':'clip_000031', 'labels':'force_bipolar,nan,out_of_view,cadiere_forceps'},
    {'from': 'clip_000060', 'to':'clip_000060', 'labels':'force_bipolar,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000089', 'to':'clip_000104', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000118', 'to':'clip_000135', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000160', 'to':'clip_000173', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000176', 'to':'clip_000176', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000177', 'to':'clip_000205', 'labels':'out_of_view,bipolar_forceps,nan,monopolar_curved_scissors'},
    {'from': 'clip_000206', 'to':'clip_000208', 'labels':'out_of_view,bipolar_forceps,nan,clip_applier'},
    {'from': 'clip_000290', 'to':'clip_000303', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000318', 'to':'clip_000334', 'labels':'out_of_view,bipolar_forceps,nan,monopolar_curved_scissors'},
    {'from': 'clip_000336', 'to':'clip_000337', 'labels':'out_of_view,bipolar_forceps,nan,vessel_sealer'},
    {'from': 'clip_000338', 'to':'clip_000343', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000395', 'to':'clip_000400', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000402', 'to':'clip_000404', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000417', 'to':'clip_000436', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000438', 'to':'clip_000441', 'labels':'bipolar_forceps,nan,clip_applier,out_of_view'},
    {'from': 'clip_000442', 'to':'clip_000461', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000518', 'to':'clip_000547', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000548', 'to':'clip_000578', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000579', 'to':'clip_000589', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000590', 'to':'clip_000604', 'labels':'out_of_view,bipolar_forceps,nan,monopolar_curved_scissors'},
    {'from': 'clip_000605', 'to':'clip_000607', 'labels':'out_of_view,clip_applier,nan,monopolar_curved_scissors'},
    {'from': 'clip_000680', 'to':'clip_000715', 'labels':'bipolar_forceps,nan,monopolar_curved_scissors,out_of_view'},
    {'from': 'clip_000716', 'to':'clip_000720', 'labels':'needle_driver,nan,needle_driver,out_of_view'},
    {'from': 'clip_000736', 'to':'clip_000739', 'labels':'out_of_view,bipolar_forceps,nan,monopolar_curved_scissors'},
    
]

for c in corrects:
    mask = (clips_df['clip_name'] >= c['from']) & (clips_df['clip_name'] <= c['to'])
    clips_df.loc[mask,'tools_present']=c['labels']


In [60]:
mask = (clips_df['clip_name'] >= 'clip_000736') & (clips_df['clip_name'] <= 'clip_000739')
clips_df[mask]

,clip_name,tools_present
736,clip_000736,"grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissors"
737,clip_000737,"grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissors"
738,clip_000738,"grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissors"
739,clip_000739,"grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissors"


We need to add a column for unique tools combination to allow intelligent data splitting where data from each split is lies in training and validation sets.

In [7]:
mapping_c = {k:v for v,k in enumerate(clips_df.tools_present.unique())}
clips_df['tools_comb_code'] = clips_df.tools_present.map(mapping_c)

Additionally, tools combination count is added to the dataframe

In [8]:
mapping_n = clips_df.tools_present.value_counts().to_dict(); mapping_n
clips_df['tools_comb_count'] = clips_df.tools_present.map(mapping_n)
print(len(clips_df))

24695


In [9]:
clips_df.head()

,clip_name,tools_present,tools_comb_code,tools_comb_count
0,clip_000000,"needle_driver,nan,needle_driver,cadiere_forceps",0,6851
1,clip_000001,"needle_driver,nan,needle_driver,cadiere_forceps",0,6851
2,clip_000002,"needle_driver,nan,needle_driver,cadiere_forceps",0,6851
3,clip_000003,"needle_driver,nan,needle_driver,cadiere_forceps",0,6851
4,clip_000004,"needle_driver,nan,needle_driver,cadiere_forceps",0,6851


Exploring the video clip counts by each combination of tools.

In [10]:
clips_df.tools_present.value_counts()

needle_driver,nan,needle_driver,cadiere_forceps                                  6851
bipolar_forceps,nan,monopolar_curved_scissors,cadiere_forceps                    5882
bipolar_forceps,nan,monopolar_curved_scissors,grasping_retractor                 2993
needle_driver,nan,needle_driver,prograsp_forceps                                 1836
cadiere_forceps,bipolar_forceps,nan,monopolar_curved_scissors                     965
force_bipolar,nan,monopolar_curved_scissors,cadiere_forceps                       775
bipolar_forceps,nan,vessel_sealer,cadiere_forceps                                 446
needle_driver,nan,needle_driver,grasping_retractor                                405
prograsp_forceps,cadiere_forceps,nan,permanent_cautery_hook_spatula               294
grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissors                  260
cadiere_forceps,bipolar_forceps,nan,vessel_sealer                                 211
prograsp_forceps,bipolar_forceps,nan,permanent_cautery